In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize

#### Retorna a categoria do local

In [2]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    return categories_list[0]['name']

#### Informações para acessar a API do foursquare

In [3]:
CLIENT_ID = '55LWIFZLZNZPSTAZZQKJXBD11BSBCUZTZPIXWVV4RFZJP1HS' # your Foursquare ID
CLIENT_SECRET = 'RPCOWGHQY5YNQUCHQUTC5VLKEYFGZN15FSIYTK5GNXNZ301E' # your Foursquare Secret
VERSION = '20180604'

#### Informações da cidade delimitada

In [4]:
latitude_nyork = 40.730610
longitude_nyork = -73.935242
radius = 5000
LIMIT = 1000
QUERY = 'beer'

#### Coletando as informações dos locais

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?intent=checkin&query={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    QUERY, CLIENT_ID, CLIENT_SECRET, latitude_nyork, longitude_nyork, VERSION, radius, LIMIT)
            
# make the GET request
results = requests.get(url).json()

In [6]:
venues = results['response']['venues'] 
nearby_venues = json_normalize(venues)
filtered_columns = ['id', 'name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.tail(10)

,id,name,categories,lat,lng
40,4bc12c95f8219c74a30ab310,Connor/Fields Beer,Beer Garden,40.714464,-73.951614
41,55b88ad7498e1653a9ffaecb,"Joshua D. Beer, MD",Doctor's Office,40.765456,-73.980542
42,4fc92684e4b0a40e5e28ca2c,Beer Cart LIRR Hunterspoint Ave,Bar,40.742457,-73.946473
43,5079cb44e4b08ed647bf5587,Beer tour,Liquor Store,40.721420,-73.957562
44,5217e0dd11d2b33a2d55a11c,Carmine Street Beers,Beer Store,40.729945,-74.003728
45,5155b97de4b0d9a3f4e12a02,Union Beer Graphics Department,Office,40.715082,-73.928637
46,565cbec4498e7ec1b08e8182,Norwinds Beer Garden,Beer Garden,40.704114,-73.930618
47,4bb6b10446d4a59322b0c6c0,Beer Sandwiches,None,40.711003,-73.949211
48,4b37ce87f964a520884625e3,Duane Reade,Pharmacy,40.719128,-73.963260
49,4cb34b64aef16dcb6287c754,Good Beer NYC,Beer Store,40.727643,-73.983835


In [7]:
latitude_nyork = 40.730610
longitude_nyork = -73.935242
radius = 5000
LIMIT = 1000
QUERY = 'icecream'

In [8]:
url = 'https://api.foursquare.com/v2/venues/search?intent=checkin&query={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    QUERY, CLIENT_ID, CLIENT_SECRET, latitude_nyork, longitude_nyork, VERSION, radius, LIMIT)
            
# make the GET request
results = requests.get(url).json()

In [9]:
venues = results['response']['venues'] 
nearby_ice_venues = json_normalize(venues)
filtered_columns = ['id', 'name', 'categories', 'location.lat', 'location.lng']
nearby_ice_venues =nearby_ice_venues.loc[:, filtered_columns]
nearby_ice_venues['categories'] = nearby_ice_venues.apply(get_category_type, axis=1)
nearby_ice_venues.columns = [col.split(".")[-1] for col in nearby_ice_venues.columns]

nearby_ice_venues.tail(10)

,id,name,categories,lat,lng
40,55de5e2b498eb54e65d9daff,Van Leeuwen Artisan Ice Cream,Ice Cream Shop,40.718395,-73.961732
41,5755db82498e0ac6e78f7da4,Lizzy's Icecream Store,Ice Cream Shop,40.727291,-74.005580
42,589800799398ab157abb6fba,Blossom Ice Cream & The Poké,Hawaiian Restaurant,40.719645,-73.962121
43,5bbd1d34c21cb1002b68b8f3,Morgenstern’s Finest Ice Cream,Ice Cream Shop,40.727057,-73.999864
44,4dcdd3fae4cd130e16586c20,Van Leeuwen Ice Cream Truck - 29th,Food Truck,40.751916,-74.001773
45,4ddc274e887775ea9c3acbe9,New York Soft Serve Ice Cream Truck,Ice Cream Shop,40.742296,-73.946989
46,4f6bc036e4b0541c5fd5cc8f,Architecture Parlor and Ice Cream Studio,Office,40.728362,-73.959961
47,504b7c44e4b04421f2d08ede,Jacques Torres Ice Cream Shop,Ice Cream Shop,40.703279,-73.991463
48,553bf653498eba55ec0bdfde,Davey's Ice Cream,Ice Cream Shop,40.717053,-73.958606
49,5b849e523abcaf002c366306,Van Leeuwen Artisan Ice Cream,Ice Cream Shop,40.726191,-73.998850


In [10]:
nearby_venues = pd.concat([nearby_venues, nearby_ice_venues])

In [11]:
len(nearby_venues)

100

In [12]:
users_total = pd.DataFrame()
for venue in nearby_venues.iterrows():
    url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}&limit={}'.format(
    venue[1]['id'], CLIENT_ID, CLIENT_SECRET, VERSION, 2000)
    results = requests.get(url).json()
    if (results['response']['likes']['count'] > 0) & ('items' in results['response']['likes']):
        results_2 = results['response']['likes']['items']
        users = json_normalize(results_2)[['id', 'firstName', 'gender']]
        users['avenue_id'] = venue[1]['id']
        users['avenue_name'] = venue[1]['name']
        users['avenue_categories'] = venue[1]['categories']
        users['avenue_lat'] = venue[1]['lat']
        users['avenue_lng'] = venue[1]['lng']
        if users_total.empty:
            users_total = users
        else:
            users_total = pd.concat([users_total, users], ignore_index=True)

In [13]:
users_total.tail()

,id,firstName,gender,avenue_id,avenue_name,avenue_categories,avenue_lat,avenue_lng
18549,656675,Yoni,male,5b849e523abcaf002c366306,Van Leeuwen Artisan Ice Cream,Ice Cream Shop,40.726191,-73.99885
18550,2071057,Katie,female,5b849e523abcaf002c366306,Van Leeuwen Artisan Ice Cream,Ice Cream Shop,40.726191,-73.99885
18551,445084965,Conor,male,5b849e523abcaf002c366306,Van Leeuwen Artisan Ice Cream,Ice Cream Shop,40.726191,-73.99885
18552,26479,sarkis,male,5b849e523abcaf002c366306,Van Leeuwen Artisan Ice Cream,Ice Cream Shop,40.726191,-73.99885
18553,8675249,Adrian,male,5b849e523abcaf002c366306,Van Leeuwen Artisan Ice Cream,Ice Cream Shop,40.726191,-73.99885


In [14]:
users_total.to_csv('curtidas_cervejarias_e_sorveterias.csv', index=False)